In [1]:
import tweepy
import numpy as np
import time
import json
import os
import pandas as pd

## Get Training Data

In [2]:
DATA_ID_PATH = "../id_data/train.data.txt"   # train

FULL_TWITTER_PATH = "../full_data/data_storage/full_train.json"
Y_TRAIN_ID_PATH = "../id_data/train.label.txt"
X_TRAIN_ID_PATH = "../id_data/train.data.txt"   # train


In [3]:
def convert_replies_id_to_sorted_text(list_of_id, data_df):
    """Convert a list of tweet ids to a list of its corresponding text in data_df
    Sort by tweets original create time"""
    if type(list_of_id) is not list:
        # if input is a single id convert type
        list_of_id = [list_of_id]
    # Select dataframe based on a list of ids
    selected_df = data_df[data_df['id'].astype(str).isin(list_of_id)]
    # Sort by their created time
    sorted_df = selected_df.sort_values(by = ["created_at"])
    # Select text fields only
    text_list = sorted_df["text"].to_list()
    return text_list

In [4]:
def join_data_id_label_v2(LABEL_PATH, ID_PATH, DATA_PATH):
    # Process Labels
    if LABEL_PATH:
        # 1: Rumour
        # 0: NonRumour
        with open(LABEL_PATH, "r") as f:
            y_label = f.read().strip().split("\n") # remove next line
        y_label = pd.DataFrame(y_label, columns = ["label"])
        y_label[y_label["label"]=="rumour"] = 1
        y_label[y_label["label"]=="nonrumour"] = 0
    y_label
    

    ## Get Dataframe Id, with first id as source Id, and values as replies, not using dict since we have duplicated keys
    total_id_list = []
    with open(ID_PATH, "r") as f:
        for line in f:
            line = line.strip().split(',') # remove next line
            source_id = line[0]
            if len(line) > 1:
                # if we have replies id
                replies_id = line[1:]
            else:
                replies_id = []
            row = [source_id, replies_id]
            total_id_list.append(row)
    len(total_id_list)
    
    ## Create a dataframe containing a list of replies
    source_df = pd.DataFrame(total_id_list, columns = ['source_id', 'replies_id'])
    data_df = pd.read_json(DATA_PATH)
    source_df["reply_text_list"] = source_df["replies_id"].apply(convert_replies_id_to_sorted_text, data_df = data_df)
    source_df["source_text"] = source_df["source_id"].apply(convert_replies_id_to_sorted_text, data_df = data_df)
    source_df["label"] = y_label
    return source_df

In [5]:
join_data_id_label_v2(Y_TRAIN_ID_PATH, DATA_ID_PATH, FULL_TWITTER_PATH)

,source_id,replies_id,reply_text_list,source_text,label
0,1250219300389974016,"[1250219116993974272, 1250219437027766273, 125...",[4. Can eating garlic help prevent infection w...,[5. Can regularly rinsing your nose with salin...,0
1,554886875303780352,"[554894001946759168, 554959644125167617, 55492...","[@Telegraph How very sad., @Telegraph @Telegra...",[French police chief killed himself after #Cha...,1
2,1237901309011021825,"[1237901311439450112, 1239862502516760577]",[Infection control for suspected or confirmed ...,[Coronavirus disease (COVID-19) advice for the...,0
3,524958128392376320,"[524961934064754688, 524959028061798401, 52495...",[@WSJ Killers go berserk when cornered. Hence...,[Ottawa police confirm that there were multipl...,0
4,1239295488677085185,[1239562248990806016],[],[if the primary focus of a government isn't to...,0
...,...,...,...,...,...
1890,671181758692507648,"[671200376843067392, 671184263329538048, 67133...","[@Bipartisanism \nDesperate!, @Bipartisanism ...",[Desperate Ted Cruz Claims Planned Parenthood ...,1
1891,672513234419638273,"[672513853645717504, 672589162726297600, 67251...",[.@ABC has anyone else noticed mass shootings ...,"[""Thoughts and prayers are not enough."" Pres. ...",1
1892,553508098825261056,"[553511652587360256, 553509546602553344, 55355...","[@NBCNews bury them in their hole, @NBCNews @W...",[Police have surrounded this building where th...,0
1893,1249479605582327808,"[1249392473073487873, 1249539822915481602, 124...",[@Kirstenjoyweiss @MattFabrication @prestone85...,[],0
